In [1]:
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

In [2]:
X,y = make_classification(n_samples = 10000, n_features = 10, n_informative = 3)

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [4]:
dt = DecisionTreeClassifier(random_state = 42)
dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)

In [5]:
print("Decision Tree accuracy", accuracy_score(y_test, y_pred))

Decision Tree accuracy 0.933


# bagging

In [6]:
bag = BaggingClassifier(
    estimator = DecisionTreeClassifier(),
    n_estimators = 500,
    max_samples = 0.5,
    bootstrap = True,
    random_state = 42
)

In [7]:
bag.fit(X_train, y_train)

BaggingClassifier(estimator=DecisionTreeClassifier(), max_samples=0.5,
                  n_estimators=500, random_state=42)

In [8]:
BaggingClassifier(estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features=None,
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    random_state=None,
                                                    splitter='best'),
                  bootstrap=True, bootstrap_features=False, max_features=1.0,
                  max_samples=0.5, n_estimators=500, n_jobs=None,
                  oob_score=False, random_state=42, verbose=0,
                  warm_start=False)

BaggingClassifier(estimator=DecisionTreeClassifier(), max_samples=0.5,
                  n_estimators=500, random_state=42)

In [9]:
y_pred = bag.predict(X_test)

In [10]:
accuracy_score(y_test, y_pred)

0.958

In [11]:
bag.estimators_samples_[0].shape

(4000,)

In [12]:
bag.estimators_features_[0].shape

(10,)

# Bagging using SVM

In [13]:
bag = BaggingClassifier(
    estimator = SVC(),
    n_estimators = 500,
    max_samples = 0.25,
    bootstrap = True,
    random_state = 42
)

In [14]:
bag.fit(X_train, y_train)
y_pred = bag.predict(X_test)
print("Bagging using SVM", accuracy_score(y_test, y_pred))

Bagging using SVM 0.924


# pasting

In [15]:
bag = BaggingClassifier(
    estimator = DecisionTreeClassifier(),
    n_estimators = 500,
    max_samples = 0.25,
    bootstrap = False,
    random_state = 42,
    verbose = 1,
    n_jobs = 1
)

In [16]:
bag.fit(X_train, y_train)
y_pred = bag.predict(X_test)
print("Pasting Classifier", accuracy_score(y_test, y_pred))

Pasting Classifier 0.9525


# Random Subspaces

In [17]:
bag = BaggingClassifier(
    estimator = DecisionTreeClassifier(),
    n_estimators = 500,
    max_samples = 1.0,
    bootstrap = False,
    max_features = 0.5,
    bootstrap_features = True,
    random_state = 42
)

In [18]:
bag.fit(X_train, y_train)
y_pred = bag.predict(X_test)
print("Random Subspaces Calssifier", accuracy_score(y_test, y_pred))

Random Subspaces Calssifier 0.9445


In [19]:
bag.estimators_samples_[0].shape

(8000,)

In [20]:
bag.estimators_features_[0].shape

(5,)

# Random Patches

In [27]:
bag = BaggingClassifier(
    estimator = DecisionTreeClassifier(),
    n_estimators = 500,
    max_samples = 0.25,
    bootstrap = True,
    max_features = 0.5,
    bootstrap_features = True,
    random_state = 42
)

In [29]:
bag.fit(X_train, y_train)
y_pred = bag.predict(X_test)
print("Random Patches Classifier", accuracy_score(y_test, y_pred))

Random Patches Classifier 0.9395


# OOB Score

In [30]:
bag = BaggingClassifier(
    estimator = DecisionTreeClassifier(),
    n_estimators = 500,
    max_samples = 0.25,
    bootstrap = True,
    oob_score = True,
    random_state = 42
)

In [31]:
bag.fit(X_train, y_train)

BaggingClassifier(estimator=DecisionTreeClassifier(), max_samples=0.25,
                  n_estimators=500, oob_score=True, random_state=42)

In [32]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

model = BaggingClassifier(
    estimator=DecisionTreeClassifier(
        ccp_alpha=0.0,
        class_weight=None,
        criterion='gini',
        max_depth=None,
        max_features=None,
        max_leaf_nodes=None,
        min_impurity_decrease=0.0,  # keep this
        min_samples_leaf=1,
        min_samples_split=2,
        min_weight_fraction_leaf=0.0,
        # presort='deprecated',  ← you can remove this too (also deprecated)
        random_state=None,
        splitter='best'
    ),
    bootstrap=True,
    bootstrap_features=False,
    max_features=1.0,
    max_samples=0.25,
    n_estimators=500,
    n_jobs=None,
    oob_score=True,
    random_state=42,
    verbose=0,
    warm_start=False
)


In [33]:
bag.oob_score_

0.956125

In [34]:
y_pred = bag.predict(X_test)
print("Accuracy", accuracy_score(y_test, y_pred))

Accuracy 0.9495


# Bagging Tips

### Bagging generally gives better results than Pasting
### Good results come around the 25% to 50% row sampling mark
### Random patches and subspaces should be used while dealing with high dimensional data
### To find the correct hyperparameter values we can do GridSearchCV/RandomSearchCV

# Applying GridSearchCV

In [36]:
from sklearn.model_selection import GridSearchCV

In [41]:
parameters = {
    'n_estimators': [50, 100, 5000],
    'max_samples': [0.2, 0.4, 0.7, 1.0],
    'bootstrap': [True, False],
    'max_features': [0.1, 0.4, 0.7, 1.0]
}

In [42]:
search = GridSearchCV(BaggingClassifier(), parameters, cv = 5)

In [ ]:
search.fit(X_train, y_train)

In [ ]:
GridSearchCV(cv=5, error_score=nan,
             estimator=BaggingClassifier(base_estimator=None, bootstrap=True,
                                         bootstrap_features=False,
                                         max_features=1.0, max_samples=1.0,
                                         n_estimators=10, n_jobs=None,
                                         oob_score=False, random_state=None,
                                         verbose=0, warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'bootstrap': [True, False],
                         'max_features': [0.1, 0.4, 0.7, 1.0],
                         'max_samples': [0.1, 0.4, 0.7, 1.0],
                         'n_estimators': [50, 100, 500]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [ ]:
search.best_params_
search.best_score_

In [ ]:
{'bootstrap': True,
 'max_features': 0.7,
 'max_samples': 0.4,
 'n_estimators': 500}